In [ ]:
#https://colab.research.google.com/drive/1yFphU6PW9Uo6lmDly_ud9a6c4RCYlwdX#scrollTo=j4DKDtFwiF0S
#http://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
#https://github.com/arushiprakash/MachineLearning/blob/main/BERT%20Word%20Embeddings.ipynb

In [1]:
!pip install transformers==3
!pip install sentence_transformers

     |████████████████████████████████| 757kB 15.7MB/s 
     |████████████████████████████████| 1.2MB 29.2MB/s 
     |████████████████████████████████| 3.0MB 51.1MB/s 
     |████████████████████████████████| 890kB 49.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7744269486632bf82b2a99bcc87db28b5bd1e125bf08f6b25912a64dd64fd8cf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 1.8MB 17.4MB/s 
     |████████████████████████████████| 3.2MB 25.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=d86488cc8df6f197f48d38efe2cfda3d0325f09831155dde21c3eab3c1016dac
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
Successfully built se

In [2]:
#http://mccormickml.com/2019/07/22/BERT-fine-tuning/
#https://github.com/abhimishra91/pytorch-tutorials/blob/master/lecture_08_a.ipynb
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [4]:
# Importing stock ml libraries
import numpy as np
import pandas as pd


import transformers
import torch
from transformers import BertTokenizer, BertModel, BertConfig
from sentence_transformers import SentenceTransformer

In [5]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:05<00:00, 78.8MB/s]


In [6]:
raw_data_path = '/content/drive/My Drive/NLP/ResearchArticlesClassification/Dataset/raw_train.csv'
#destination_folder = '/content/drive/My Drive/NLP/ResearchArticlesClassification/Dataset'

df_raw =  pd.read_csv(raw_data_path, encoding='utf8')

df_raw.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


In [7]:
df_raw.shape

(20972, 9)

In [8]:
document_embeddings = sbert_model.encode(df_raw['TITLE'])

In [9]:
document_embeddings.shape

(20972, 768)

In [11]:
document_embeddings

array([[-0.5190686 , -0.582919  ,  1.8336611 , ..., -0.50238967,
        -0.51798123,  0.505846  ],
       [ 0.28318253, -0.94039226,  1.7706267 , ..., -0.72228265,
        -1.2445506 ,  0.18752617],
       [-0.44547495, -0.53956467,  1.3178202 , ...,  0.24702364,
        -0.70157164,  0.5005233 ],
       ...,
       [-0.01401247,  0.47100332,  0.71465963, ...,  0.22713855,
         0.19967219,  0.8573516 ],
       [-0.8528395 , -0.37673137,  1.1304436 , ...,  0.01878909,
        -0.56280094,  0.39647996],
       [-0.3008775 ,  0.24345447,  1.1184505 , ..., -0.45542303,
        -1.159588  ,  0.02022636]], dtype=float32)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [13]:
pairwise_similarities = cosine_similarity(document_embeddings)

In [14]:
len(pairwise_similarities)

20972

In [29]:
query = 'computer vius'
query_embeddings = sbert_model.encode(query)

In [30]:
query_embeddings.shape

(768,)

In [ ]:
with open('/content/drive/My Drive/NLP/ResearchArticlesClassification/title_embeddings.npy', 'wb') as f:
    np.save(f, document_embeddings)

In [31]:
from sentence_transformers import SentenceTransformer, util
cos_scores = util.pytorch_cos_sim(query_embeddings, document_embeddings)[0]
top_results = torch.topk(cos_scores, k=3)


In [32]:
top_results

torch.return_types.topk(values=tensor([0.7427, 0.7269, 0.7204]), indices=tensor([13308, 19721,   313]))

In [43]:
for i in top_results[1]:
  print(i)
  print(i.item())

tensor(13308)
13308
tensor(19721)
19721
tensor(313)
313


In [36]:
df_raw.iloc[13308]

ID                                                                  13309
TITLE                                             Machines and Algorithms
ABSTRACT                  I discuss the evolution of computer architec...
Computer Science                                                        1
Physics                                                                 1
Mathematics                                                             0
Statistics                                                              0
Quantitative Biology                                                    0
Quantitative Finance                                                    0
Name: 13308, dtype: object

In [44]:
print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
  print(df_raw.iloc[idx.item()]['TITLE'])
  print("(Score: {:.4f})".format(score.item()))





Query: computer vius

Top 5 most similar sentences in corpus:
Machines and Algorithms
(Score: 0.7427)
From Attention to Participation: Reviewing and Modelling Engagement with Computers
(Score: 0.7269)
Human perception in computer vision
(Score: 0.7204)


In [69]:
def search_project_title(query_term, all_title_embeddings, topn):
 # query_embedding = np.array((768))
  query_embedding = sbert_model.encode(query)
  #print(type(query_embeddings))
  cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]
  top_results = torch.topk(cos_scores, k= topn)

  print("Query:", query)
  print("\nTop {} most similar sentences in corpus:".format(topn))

  for score, idx in zip(top_results[0], top_results[1]):
    print(df_raw.iloc[idx.item()]['TITLE'], ":"  , "(Score: {:.4f})".format(score.item()))

In [72]:
query = 'Valid Inference Corrected for Outlier Removal'
search_project_title(query, document_embeddings, 5)

Query: Valid Inference Corrected for Outlier Removal

Top 5 most similar sentences in corpus:
Valid Inference Corrected for Outlier Removal : (Score: 1.0000)
Efficient boundary corrected Strang splitting : (Score: 0.8487)
Efficient Estimation for Dimension Reduction with Censored Data : (Score: 0.8367)
Optimal stopping via reinforced regression : (Score: 0.8279)
Outlier Detection by Consistent Data Selection Method : (Score: 0.8267)


In [15]:
#a = pairwise_similarities[0][pairwise_similarities[0] > 0.65]

In [16]:
def most_similar(doc_id, similarity_matrix, matrix, topn, confidence_level=0.75):
    title_name = df_raw.iloc[doc_id]["TITLE"]
    print (f'Document: {title_name}')
    print ('\n')
    print (f'Similar Projects for {title_name}:')
    if matrix=='Cosine Similarity':
        #a = pairwise_similarities[doc_id][pairwise_similarities[doc_id] > confidence_level]
        #print(a)
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
      
    for ix in similar_ix[:topn + 1]:
        if ix==doc_id:
            continue
        #print (similarity_matrix[doc_id][ix])
        a = similarity_matrix[doc_id][ix]
        if a > confidence_level:
          print (f'Document: {df_raw.iloc[ix]["TITLE"]}')
          print (f'{matrix} : {similarity_matrix[doc_id][ix]}\n')
          

In [19]:
most_similar(120,pairwise_similarities,'Cosine Similarity', 10, 0.75)

Document: Fault Tolerant Consensus Agreement Algorithm


Similar Projects for Fault Tolerant Consensus Agreement Algorithm:
Document: Rate-optimal Meta Learning of Classification Error
Cosine Similarity : 0.8287526965141296

Document: Outlier Detection by Consistent Data Selection Method
Cosine Similarity : 0.8206487894058228

Document: Guaranteed Fault Detection and Isolation for Switched Affine Models
Cosine Similarity : 0.8133232593536377

Document: Analysis of Distributed ADMM Algorithm for Consensus Optimization in Presence of Error
Cosine Similarity : 0.8038773536682129

Document: Valid Inference Corrected for Outlier Removal
Cosine Similarity : 0.8017244338989258

Document: Discrete Choice and Rational Inattention: a General Equivalence Result
Cosine Similarity : 0.7992138862609863

Document: Efficient Estimation for Dimension Reduction with Censored Data
Cosine Similarity : 0.7958940267562866

Document: Efficient Structured Surrogate Loss and Regularization in Structured Predic